<a href="https://colab.research.google.com/github/AmitPeled/ComputerVisionProject/blob/main/instant_ngp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/ceres-solver-v2.zip
!unzip /content/ceres-solver-v2.zip -d ceres-solver

!cp -r /content/ceres-solver/lib/. /usr/local/lib
!chmod 755 /content/ceres-solver/bin/colmap
!cp -r /content/ceres-solver/bin/. /usr/local/bin

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/instant-ngp-v2.zip
!unzip /content/instant-ngp-v2.zip -d instant-ngp

%cd /content/instant-ngp

import os

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-filesystem-dev is already the newest version (1.74.0.3ubuntu7).
libboost-filesystem-dev set to manually installed.
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libatlas-base-dev is already the newest version (3.10.3-12ubuntu1).
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-graph-dev set to manually installed.
libboost-test-dev is already the newest version (1.74.0.3ubuntu7).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libegl-dev libevde

In [2]:
scene_path = "/content/drive/MyDrive/COLMAP_Project/frames" #@param {type:"string"}
colmap_path = "/content/drive/MyDrive/COLMAP_Project/sparse_txt" #@param {type:"string"}
train_path = "/content/train"
if not os.path.isdir(scene_path) or not os.path.isdir(scene_path):
  raise NotADirectoryError()
!rm -rf {train_path}
!mkdir {train_path}
!cp -r {scene_path}/. /content/train
transforms_path = os.path.join(train_path, f"transforms.json")
!cp {colmap_path}/transforms.json {transforms_path}
#!python ./scripts/colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 4 --images {train_path} --out {transforms_path}

train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(f'{colmap_path}/instantngp', f"{train_steps}.msgpack")

!pip install commentjson
!python ./scripts/run.py --scene {train_path} --mode nerf --n_steps {train_steps} --save_snapshot {snapshot_path}

Traceback (most recent call last):
  File "/content/instant-ngp/./scripts/run.py", line 13, in <module>
    import commentjson as json
ModuleNotFoundError: No module named 'commentjson'


In [ ]:
!pip install commentjson huggingface-hub
from huggingface_hub import create_repo, upload_folder

hf_token = "" #@param {type:"string"}
repo_id = "camenduru/instant-ngp-models" #@param {type:"string"}
path_in_repo = "fox" #@param {type:"string"}
create_repo(repo_id, private=True, token=hf_token)
upload_folder(folder_path=train_path, path_in_repo=path_in_repo, repo_id=repo_id, commit_message=f"{scene_path}", token=hf_token)